## Introduction to `robotoc` 
# 3: Optimal Control of a Robot Manipulator

## The goal of this chapter
Here, we learn how to construct and solve an optimal control problem (OCP) for a robot manipulator KUKA iiwa14.

First, we construct `robotoc.Robot` of a robot manipulator KUKA iiwa14.

In [ ]:
import robotoc
model_info = robotoc.RobotModelInfo()
model_info.urdf_path = "urdf/iiwa_description/urdf/iiwa14.urdf"
iiwa14 = robotoc.Robot(model_info)

Here, we change the joint torque and velocity limits of the robot manipulator.  
Like the following, you can change such limits depending on your goal.

In [ ]:
import numpy as np
iiwa14.set_joint_effort_limit(np.full(iiwa14.dimu(), 50))
iiwa14.set_joint_velocity_limit(np.full(iiwa14.dimv(), 0.5*np.pi))

You can check that these limits are set in the robot model.

In [ ]:
print(iiwa14)

## Cost Function
Next, we create the cost function of the optimal control problem (OCP).   
In `robotoc`, we create the cost function `robotoc.CostFunction` (`std::shared_ptr<robotoc::CostFunction>` in C++) by combining various **cost components**.  
That is, `robotoc.CostFunction` (`std::shared_ptr<robotoc::CostFunction>` in C++) is a collection of the **cost components** .   
Each **cost component** inherits `robotoc.CostFunctionComponentBase` (`robotoc::CostFunctionComponentBase` in C++).    
By doing so, we can easily combine many kinds of **cost components**.

Basic and useful **cost components** are already implemented in robotoc.  
In this example, we utilize the **cost component on the configuration (i.e., cost on the joint-space)** as 

In [ ]:
config_cost = robotoc.ConfigurationSpaceCost(iiwa14)

We set the reference values and weight parameters of the configuration space cost compoonent.  
Naming conventions are as follows:
- `q_ref`: reference configuration (joint positions)
- `q_weight`: weight on the configuration 
- `v_weight`: weight on the generalized velocity (joint velocities)
- `a_weight`: weight on the generalized acceleration (joint accelerations)
- `u_weight`: weight on the joint torques
- `q_weight_terminal`, `v_weight_terminal`: weights on the configuration and generalized velocity at the terminal time of the horizon

In [ ]:
q_ref = np.array([0, 0.5*np.pi, 0, 0.5*np.pi, 0, 0.5*np.pi, 0]) 
config_cost.set_q_ref(q_ref)
config_cost.set_q_weight(np.full(iiwa14.dimv(), 10))
config_cost.set_v_weight(np.full(iiwa14.dimv(), 0.01))
config_cost.set_a_weight(np.full(iiwa14.dimv(), 0.01))
config_cost.set_q_weight_terminal(np.full(iiwa14.dimv(), 10))
config_cost.set_v_weight_terminal(np.full(iiwa14.dimv(), 0.01))

We then make the **cost function** `robotoc.CostFunction` and add the **configuration cost component** to it.

In [ ]:
cost = robotoc.CostFunction()
cost.push_back(config_cost)

## Constraints 
We can construct constraints in the same way as the cost function.  
In `robotoc`, we create the constraints `robotoc.Constraints` (`std::shared_ptr<robotoc::Constraints>` in C++) by combining various **constraint components**.  
That is, the `robotoc.Constraints` (`std::shared_ptr<robotoc::Constraints>` in C++) is a collection of the **constraint components** .  
Each **constraints component** inherits `robotoc.ConstraintComponentBase` (`robotoc::ConstraintComponentBase` in C++).   
By doing so, we can easily combine many kinds of **constraint components**.

Basic and useful **constraint components** are already implemented in robotoc.  
In this example, we utilize **constraint components on the joint position, velocity, and torque limits** as   
(limits set by `iiwa14.set_joint_effort_limit()` `iiwa14.set_joint_velocity_limit()` are used in these constraint components)

In [ ]:
joint_position_lower  = robotoc.JointPositionLowerLimit(iiwa14)
joint_position_upper  = robotoc.JointPositionUpperLimit(iiwa14)
joint_velocity_lower  = robotoc.JointVelocityLowerLimit(iiwa14)
joint_velocity_upper  = robotoc.JointVelocityUpperLimit(iiwa14)
joint_torques_lower   = robotoc.JointTorquesLowerLimit(iiwa14)
joint_torques_upper   = robotoc.JointTorquesUpperLimit(iiwa14)

We then make the **constraints** `robotoc.Constraints` and add these **constraint components**.

In [ ]:
constraints = robotoc.Constraints()
constraints.push_back(joint_position_lower)
constraints.push_back(joint_position_upper)
constraints.push_back(joint_velocity_lower)
constraints.push_back(joint_velocity_upper)
constraints.push_back(joint_torques_lower)
constraints.push_back(joint_torques_upper)

## Optimal Control Problem (OCP) and Solver

Now we can construct the optimal control problem (OCP) and solvers.  
First, set the length of the horizon `T` and the number of discretization grids `N` and construct the optimal control problem (OCP) `robotoc.OCP` (`robotoc::OCP` in C++) as

In [ ]:
T = 3.0
N = 100
ocp = robotoc.OCP(robot=iiwa14, cost=cost, constraints=constraints, T=T, N=N)

For this OCP, first we try `robotoc.UnconstrOCPSolver` (`robotoc::UnconstrOCPSolver` in C++).  
This solver has the following characteristics: 
- Direct multiple-shooting method and primal-dual interior point method
- Inverse dynamics-based solution method for **unconstrained OCPs (i.e., no contacts or floating bases)**.
- Riccati recursion to compute the Newton-steps

We can specify some solver options by `robotoc.SolverOptions` (we use the default settings in the following) such as `nthreads`, the number of threads used in parallel computing with the direct multiple shooting method.

In [ ]:
solver_options = robotoc.SolverOptions()
solver_options.nthreads = 4
ocp_solver = robotoc.UnconstrOCPSolver(ocp=ocp, solver_options=solver_options)

## Solve the OCP

We solve the OCP in which the initial state (`q`, `v`) and initial time (`t`) are given as follows.

In [ ]:
t = 0.0
q = np.array([0.5*np.pi, 0, 0.5*np.pi, 0, 0.5*np.pi, 0, 0.5*np.pi]) 
v = np.zeros(iiwa14.dimv())

We initialize the OCP solver.  
The following appropriate initialization is very simple but important to solve the OCP efficiently.   
First, we set the initial guess of the solution as the above `q` and `v`.   
Before set the initial guesses, we discretize the OCP by `discretize(t)` method.

In [ ]:
ocp_solver.discretize(t)
ocp_solver.set_solution("q", q)
ocp_solver.set_solution("v", v)

We also initialize the solver to treat the inequality constraints.   
The following methods do it. (Specifically, it initializes slack and dual variables of the primal-dual interior point method).

In [ ]:
ocp_solver.init_constraints()

To see how the current iterate is close to the stationary points, we see the **KKT error**, which is an l2-norm of the Karush–Kuhn–Tucker (KKT) conditions (first-order necessary conditions of the optimality).   
After the above initialization, the KKT error is 

In [ ]:
ocp_solver.KKT_error(t, q, v)

Now we solve the OCP

In [ ]:
ocp_solver.solve(t, q, v)

We can see that the KKT error is very small. (Specifically, smaller than the threshold set in `robotoc.SolverOption`).

In [ ]:
ocp_solver.KKT_error(t, q, v)

We can also see the stats of the solver as follows.

In [ ]:
stats = ocp_solver.get_solver_statistics()
print(stats)

Next, we plot the solution trajectory. We can get he solution from the solver by

In [ ]:
q_opt = ocp_solver.get_solution('q')
v_opt = ocp_solver.get_solution('v')
u_opt = ocp_solver.get_solution('u')

From the plot, we can see that the solution strictly satisfies the constraints, which are plotted as dashed lines

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn

plt.rcParams['figure.figsize'] = 10, 9
fig = plt.figure()
ax_q = fig.add_subplot(3, 1, 1)
ax_v = fig.add_subplot(3, 1, 2)
ax_u = fig.add_subplot(3, 1, 3)

# Plot results
ax_q.plot(np.linspace(t, t+T, N+1), q_opt)
ax_v.plot(np.linspace(t, t+T, N+1), v_opt)
ax_u.plot(np.linspace(t, t+T-(T/N), N), u_opt)

# Plot constraints boundaries
ax_v.plot(np.linspace(t, t+T, N+1), np.full(N+1, 0.5*np.pi), linestyle='--', color='gray')
ax_v.plot(np.linspace(t, t+T, N+1), np.full(N+1, -0.5*np.pi), linestyle='--', color='gray')
ax_u.plot(np.linspace(t, t+T-(T/N), N), np.full(N, 50.0), linestyle='--', color='gray')
ax_u.plot(np.linspace(t, t+T-(T/N), N), np.full(N, -50.0), linestyle='--', color='gray')

ax_q.set_xlabel('')
ax_v.set_xlabel('')
ax_u.set_xlabel('t [s]')

ax_q.set_ylabel('q [rad]')
ax_v.set_ylabel('v [rad/s]')
ax_u.set_ylabel('u [Nm]')

We can also visualize the solution trajectory.

In [ ]:
viewer = robotoc.utils.TrajectoryViewer(model_info=model_info,
                                        viewer_type='meshcat')
viewer.display(ocp_solver.get_time_discretization(), 
               ocp_solver.get_solution('q'))

## Try Another Solver

Next, we try another solver `robotoc.UnconstrParNMPCSolver` (`robotoc::UnconstrParNMPCSolver` in C++).  
This solver has the following characteristics: 
- Direct multiple-shooting method and primal-dual interior point method
- Inverse dynamics-based solution method for **unconstrained** OCPs (i.e., no contacts or floating bases).
- ParNMPC algorithm (highly parallelizable Newton-type method) to compute the Newton-steps
- Each Newton-step computation is very fast if ta large number of processors are available.
- Convergence speed (i.e., required number of iterations until convergence) is worse than `robotoc.UnconstrOCPSolver`. It also means that the solver sometimes lacks robustness.

We can specify some solver options by `robotoc.SolverOptions` (we use the default settings in the following) including `nthreads`, the number of threads used in parallel computing.

In [ ]:
solver_options = robotoc.SolverOptions()
solver_options.nthreads = 8
parnmpc_solver = robotoc.UnconstrParNMPCSolver(ocp=ocp, solver_options=solver_options)

In [ ]:
parnmpc_solver.discretize(t)
parnmpc_solver.set_solution("q", q)
parnmpc_solver.set_solution("v", v)

In [ ]:
parnmpc_solver.init_constraints()
parnmpc_solver.init_backward_correction()
parnmpc_solver.solve(t, q, v)
stats = parnmpc_solver.get_solver_statistics()
print(stats)